In [5]:

from google.colab import drive
drive.mount('/content/drive')

import os, shutil, pathlib, tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

IMG_SIZE = (224, 224)
BATCH = 32
SEED = 1337

# 1) Tente achar um dataset no seu Drive com >=2 subpastas de classe
CANDIDATES = [
    "/content/drive/MyDrive/tl_data",           # recomendado
    "/content/drive/MyDrive/mydriveface",     # sua pasta antiga
    "/content/drive/MyDrive/dataset_recomendacao"
]
def has_two_classes(root):
    p = pathlib.Path(root)
    if not p.exists(): return False
    sub = [d for d in p.iterdir() if d.is_dir()]
    cls = 0
    for d in sub:
        n = sum(1 for x in d.rglob('*') if x.suffix.lower() in {'.jpg','.jpeg','.png'})
        if n > 0: cls += 1
    return cls >= 2

ds_root = None
for c in CANDIDATES:
    if has_two_classes(c):
        ds_root = c; break

# 2) Se não achou 2 classes, baixa exemplo (roses vs tulips)
if ds_root is None:
    print("⚠️ Não encontrei dataset com 2 classes no Drive. Baixando exemplo (roses × tulips)...")
    url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
    path = tf.keras.utils.get_file("flower_photos", origin=url, untar=True)
    base = pathlib.Path(path)
    tmp = "/content/tl_data_flowers"
    os.makedirs(tmp, exist_ok=True)
    for cls in ["roses", "tulips"]:
        shutil.copytree(base/cls, pathlib.Path(tmp)/cls, dirs_exist_ok=True)
    ds_root = tmp

print("📂 Usando dataset em:", ds_root)

# 3) Datasets (80/20) — binário
train_ds = tf.keras.utils.image_dataset_from_directory(
    ds_root, validation_split=0.2, subset="training", seed=SEED,
    image_size=IMG_SIZE, batch_size=BATCH, label_mode="binary")

val_ds = tf.keras.utils.image_dataset_from_directory(
    ds_root, validation_split=0.2, subset="validation", seed=SEED,
    image_size=IMG_SIZE, batch_size=BATCH, label_mode="binary")

print("Classes:", train_ds.class_names)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)

# 4) Modelo pré-treinado (MobileNetV2) + cabeça nova
base = MobileNetV2(input_shape=IMG_SIZE+(3,), include_top=False, weights="imagenet")
base.trainable = False

inp = layers.Input(shape=IMG_SIZE+(3,))
x = preprocess_input(inp)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
out = layers.Dense(1, activation="sigmoid")(x)  # 2 classes

model = models.Model(inp, out)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# 5) Treino rápido
history = model.fit(train_ds, validation_data=val_ds, epochs=5)

# 6) Teste com upload de 1 imagem
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np

print("Envie uma imagem para testar…")
up = files.upload()
img_path = list(up.keys())[0]
img = image.load_img(img_path, target_size=IMG_SIZE)
arr = image.img_to_array(img)[None, ...]
arr = preprocess_input(arr)
p = float(model.predict(arr, verbose=0)[0][0])
pred = train_ds.class_names[int(p > 0.5)]
print(f"Predição: {pred}  |  score={p:.3f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⚠️ Não encontrei dataset com 2 classes no Drive. Baixando exemplo (roses × tulips)...


FileNotFoundError: [Errno 2] No such file or directory: '/root/.keras/datasets/flower_photos/roses'